# Generate Website

This notebook generates the website for the dataset.

In [1]:
%reload_ext autoreload
%autoreload 2

## Generate JSON Reports

In [29]:
from depsurf import DepKind, VersionGroup, DepReport, Dep
from utils import REPORT_PATH


def get_report_path(dep: Dep):
    return REPORT_PATH / dep.kind / f"{dep.name}.json"


DEPS = [
    DepKind.FUNC("__blk_account_io_done"),
    DepKind.FUNC("__page_cache_release"),
    DepKind.STRUCT("task_struct"),
    DepKind.STRUCT("fs_context"),
    DepKind.STRUCT("inode"),
    # DepKind.FUNC("vfs_rename"),
    # DepKind.FUNC("init_once"),
    DepKind.FUNC("vfs_read"),
]

# for dep in DEPS:
#     report = DepReport.from_groups(
#         dep, [VersionGroup.REGULAR, VersionGroup.ARCH, VersionGroup.FLAVOR]
#     )
#     report.dump_json(get_report_path(dep))

# DepReport.from_dump(get_report_path(DEPS[-1]))

In [1]:
KINDS = [
    # DepKind.FUNC,
    # DepKind.STRUCT,
    DepKind.LSM,
]

GROUPS = [VersionGroup.REGULAR, VersionGroup.ARCH, VersionGroup.FLAVOR]

DEPS = sorted(
    set(
        kind(name)
        for group in GROUPS
        for v in group
        for kind in KINDS
        for name in v.img.get_all_by_kind(kind).keys()
    )
)

for dep in DEPS:
    report = DepReport.from_groups(dep, GROUPS)
    report.dump_json(get_report_path(dep))

[        types.py:19 ] INFO: Loading types from /Users/szhong/Code/DepSurf/data/dataset/types_struct/4.4.0-21-generic-amd64.jsonl
[        types.py:19 ] INFO: Loading types from /Users/szhong/Code/DepSurf/data/dataset/types_func/4.4.0-21-generic-amd64.jsonl
[        types.py:19 ] INFO: Loading types from /Users/szhong/Code/DepSurf/data/dataset/types_struct/4.8.0-22-generic-amd64.jsonl
[        types.py:19 ] INFO: Loading types from /Users/szhong/Code/DepSurf/data/dataset/types_func/4.8.0-22-generic-amd64.jsonl
[        types.py:19 ] INFO: Loading types from /Users/szhong/Code/DepSurf/data/dataset/types_struct/4.10.0-19-generic-amd64.jsonl
[        types.py:19 ] INFO: Loading types from /Users/szhong/Code/DepSurf/data/dataset/types_func/4.10.0-19-generic-amd64.jsonl
[        types.py:19 ] INFO: Loading types from /Users/szhong/Code/DepSurf/data/dataset/types_struct/4.13.0-16-generic-amd64.jsonl
[        types.py:19 ] INFO: Loading types from /Users/szhong/Code/DepSurf/data/dataset/types

## Generate Markdown Reports

In [28]:
from depsurf import DepReport, DepKind
from utils import WEBSITE_PATH, REPORT_PATH

KINDS = [
    DepKind.FUNC,
    DepKind.STRUCT,
    # DepKind.LSM,
]


for kind in KINDS:
    paths = (REPORT_PATH / kind).rglob("*.json")
    for json_path in paths:
        report = DepReport.from_dump(json_path)
        path = (
            WEBSITE_PATH
            / report.dep.kind
            / report.dep.name.replace("_", "").lower()[0]
            / f"{report.dep.name}.md"
        )
        report.dump_md(path)

## Generate Index Pages

In [3]:
from typing import Iterable
from pathlib import Path

from depsurf import DepKind
from utils import WEBSITE_PATH, WEBSITE_URL


KINDS = [
    DepKind.FUNC,
    DepKind.STRUCT,
    DepKind.LSM,
]

LARGE_KINDS = [DepKind.FUNC, DepKind.STRUCT]


def print_index(dep_paths: Iterable[Path], file_path: Path):
    with open(file_path, "w") as f:
        for path in sorted(dep_paths, key=lambda d: d.stem.replace("_", "").lower()):
            url = path.relative_to(WEBSITE_PATH).with_suffix(".html")
            print(f"[`{path.stem}`]({WEBSITE_URL}/{url})\n", file=f)


for kind in KINDS:
    kind_path = WEBSITE_PATH / str(kind)
    subdirs = [d for d in kind_path.iterdir() if d.is_dir()]
    if kind in LARGE_KINDS:
        for subdir in subdirs:
            print_index(
                dep_paths=subdir.glob("*.md"),
                file_path=kind_path / f"{subdir.stem}.md",
            )
    else:
        print_index(
            dep_paths=[p for d in subdirs for p in d.glob("*.md")],
            file_path=kind_path / "index.md",
        )


In [5]:
from depsurf import IssueEnum, VersionGroup

from collections import defaultdict

partitioned_results = defaultdict(dict)

for report in FUNC_REPORTS:
    result = {}
    for (group, version), issues in report.issue_dict.items():
        v = version.short_version
        if IssueEnum.FULL_INLINE in issues:
            result[v] = "F"
        elif IssueEnum.SELECTIVE_INLINE in issues:
            result[v] = "S"
        elif IssueEnum.ABSENT in issues:
            result[v] = "-"
        else:
            result[v] = ""

    dep = report.dep
    key = dep.name.replace("_", "").lower()
    partitioned_results[key[:1]][(key, dep.name)] = result


partitioned_results = {k: sorted(v.items()) for k, v in partitioned_results.items()}

In [6]:
INLINED_PATH = WEBSITE_PATH / "inline"
INLINED_PATH.mkdir(exist_ok=True)

for char, results in partitioned_results.items():
    with open(INLINED_PATH / f"{char}.html", "w") as f:
        f.write("<html>\n")
        f.write("<head>\n")
        f.write(f"<title>Kernel Function Inline Status - {char}</title>\n")
        f.write('<link rel="stylesheet" href="style.css">\n')
        f.write("</head>\n")

        f.write("<body>\n")
        f.write("<table>\n")
        f.write("<thead>\n")

        f.write("<tr>\n")
        f.write(
            f'<td><span class="f">&nbsp;F&nbsp;</span>: Fully Inlined. <span class="p">&nbsp;P&nbsp;</span>: Partially Inlined. <span class="m">&nbsp;-&nbsp;</span>: Absent</td>\n'
        )
        for version in VersionGroup.REGULAR:
            f.write(f"<th>{version.short_version}</th>")
        f.write("</tr>\n")
        f.write("</thead>\n")

        f.write("<tbody>\n")
        for (_, dep), result in results:
            f.write(f'<tr><th id="{dep}"><pre>{dep}</pre></th>')
            for v in result.values():
                class_name = {
                    "F": "f",
                    "P": "p",
                    "-": "m",
                }.get(v)
                if class_name is not None:
                    f.write(f'<td class="{class_name}">{v}</td>')
                else:
                    f.write(f"<td>{v}</td>")
            f.write("</tr>\n")
        f.write("</tbody>\n")

        f.write("</table>\n")
        f.write("</body>\n")
        f.write("</html>\n")